In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split
import time 

In [2]:
columns = ['X1', 'X2', 'Y']
df = pd.read_csv('Dataset.csv', names= columns)
df.head()

,X1,X2,Y
0,-0.016816,1.493616,-0.996794
1,-0.095445,2.742787,-1.169155
2,-1.900142,0.674756,0.256157
3,-0.982229,1.464483,0.880567
4,0.919084,1.495322,-0.863523


### Reading and splitting the data

In [3]:
x = df.drop(columns= 'Y')
x = x.to_numpy()
y = df['Y'].to_numpy()

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.25, random_state=2108602)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=2108602)

In [5]:
train_data = np.hstack((x_train, np.reshape( y_train, (-1, 1) )))

In [6]:
print(x_test.shape)
print(x_train.shape)
print(x_val.shape)

(31, 2)
(187, 2)
(32, 2)


### Input parameters

In [7]:
Ns = [3, 5, 7, 9, 12, 15] # number of centers (input parameter)
n = x.shape[1] # number of features
rhos = [1e-01, 1e-02, 1e-03, 1e-04, 1e-05]

### Initial weights and weights matrix formation

In [8]:
def init_W(N,n):
    W = np.random.random(N*n+N)
    c = np.reshape(W[0: N*n], (N, n))
    v = np.reshape(W[N*n:], (-1, 1))
    return W, c, v

### The forward propagation

In [9]:
def RBF(x, c, sigma=1):
    phi = np.exp(-(np.linalg.norm(x-c)/ sigma)**2)
    return phi

In [10]:
def y_hat(x, c, v, sigma=1):
    phi_mtx = np.zeros((len(x), N))
    for i in range(len(x)):
        for j in range(N):
            phi_mtx[i,j] = RBF(x[i], c[j])
    y_h = np.squeeze(np.dot(phi_mtx, v))
    return y_h

### define the loss function by defining the MSE and the regularized error


In [11]:
def mse(y_h, y):
    return sum((y_h - y)**2)/(2*len(y))

In [12]:
def err_fn(W, train_data):
    x_train = train_data[:, 0:2]
    y_train = train_data[:, 2]
    c = np.reshape(W[0: N*n], (N, n))
    v = np.reshape(W[N*n:], (-1, 1))
    y_h = y_hat(x_train, c, v)
    regularization_term = 0.5 * rho * (np.linalg.norm(v) + np.linalg.norm(c))
    e_mse = mse(y_h, y_train)
    return regularization_term+e_mse


In [13]:
err_out = np.zeros(len(Ns)*len(rhos))
err_test = np.zeros(len(Ns)*len(rhos)) 
Weights = np.zeros((len(Ns)*len(rhos), Ns[-1]*n+Ns[-1]))
rho_val = np.zeros(len(Ns)*len(rhos))
N_val = np.zeros(len(Ns)*len(rhos))
mse_test = np.zeros(len(Ns)*len(rhos))
mse_train = np.zeros(len(Ns)*len(rhos))
variance = np.zeros(len(Ns)*len(rhos))
cost_time = np.zeros(len(Ns)*len(rhos))
k = 0
for N in Ns:
    for rho in rhos:
        np.random.seed(2108602)
        W0 = np.random.random(N*n+N)
        Optimization_method = "L-BFGS-B"
        start = time.time()
        res = minimize(err_fn, W0, args=(train_data), method=Optimization_method, tol=1e-7)
        cost_time[k] = time.time() - start 
        err_out[k] = round(res.fun, 3)
        Weights[k, 0:N*n+N] = res.x
        W = res.x
        c = np.reshape(W[0: N*n], (N, n))
        v = np.reshape(W[N*n:], (-1, 1))
        y_pred_test = y_hat(x_test, c, v)
        mse_test[k] = round(mse(y_pred_test, y_test), 3) 
        y_pred_train = y_hat(x_train, c, v)
        mse_train[k] = round(mse(y_pred_train, y_train), 3)
        variance[k] = np.abs(round(mse_test[k] - mse_train[k], 3))
        rho_val[k] = rho
        N_val[k] = N 
        print(f"Centers: {N}, rho: {rho}, Regularized_Error: {err_out[k]}, MSE_Test: {mse_test[k]}, MSE_Train: {mse_train[k]}, Variance: {variance[k]}, Time: {cost_time[k]}") 
        k = k+1

Centers: 3, rho: 0.1, Regularized_Error: 1.067, MSE_Test: 0.432, MSE_Train: 0.976, Variance: 0.544, Time: 1.286848545074463
Centers: 3, rho: 0.01, Regularized_Error: 0.875, MSE_Test: 0.266, MSE_Train: 0.838, Variance: 0.572, Time: 3.9706778526306152
Centers: 3, rho: 0.001, Regularized_Error: 0.841, MSE_Test: 0.264, MSE_Train: 0.837, Variance: 0.573, Time: 2.503800630569458
Centers: 3, rho: 0.0001, Regularized_Error: 0.837, MSE_Test: 0.264, MSE_Train: 0.837, Variance: 0.573, Time: 2.1342155933380127
Centers: 3, rho: 1e-05, Regularized_Error: 0.837, MSE_Test: 0.264, MSE_Train: 0.837, Variance: 0.573, Time: 2.2522964477539062
Centers: 5, rho: 0.1, Regularized_Error: 1.048, MSE_Test: 0.429, MSE_Train: 0.975, Variance: 0.546, Time: 4.029079437255859
Centers: 5, rho: 0.01, Regularized_Error: 0.854, MSE_Test: 0.259, MSE_Train: 0.813, Variance: 0.554, Time: 11.190563440322876
Centers: 5, rho: 0.001, Regularized_Error: 0.814, MSE_Test: 0.245, MSE_Train: 0.809, Variance: 0.564, Time: 15.55877900

In [14]:
output_df = {
    'Rho': rho_val,
    'N_Centers': N_val,
    'MSE_Test': mse_test,
    'MSE_Train': mse_train,
    'Variance': variance,
    'Regularized_Error': err_out,
    'Time': cost_time
}

In [15]:
out_df = pd.DataFrame(output_df)
out_df.head()

,Rho,N_Centers,MSE_Test,MSE_Train,Variance,Regularized_Error,Time
0,0.10000,3.0,0.432,0.976,0.544,1.067,1.286849
1,0.01000,3.0,0.266,0.838,0.572,0.875,3.970678
2,0.00100,3.0,0.264,0.837,0.573,0.841,2.503801
3,0.00010,3.0,0.264,0.837,0.573,0.837,2.134216
4,0.00001,3.0,0.264,0.837,0.573,0.837,2.252296


In [17]:
directory = 'F:\Sapienza Courses\optimization methods for machine learning\Project_Assignment_1.2' 
file_path = directory + '\output_data_1.2.xlsx' 
out_df.to_excel(file_path, index=False)

In [ ]:
"""final_W = res.x
c = np.reshape(final_W[0: N*n], (N, n))
v = np.reshape(final_W[N*n:], (-1, 1))
print('Final Centers: ', c)
print('Final V ', v)"""

In [ ]:
# The MSE for the test, and training data
"""
y_pred_train = y_hat(x_train, c, v)
mse_train = mse(y_pred_train, y_train)
print('MSE for the training dataset: ', mse_train) 
y_pred = y_hat(x_test, c, v)
mse_test = mse(y_pred, y_test)
print('MSE for the test dataset', mse_test)"""

In [ ]:
# Plotting the output
"""
from matplotlib import projections
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(dpi=120)
x1_vals = np.linspace(-2, 2, 50)
x2_vals = np.linspace(-3, 3, 50)
x1_mesh, x2_mesh = np.meshgrid(x1_vals, x2_vals)
x_mesh = np.column_stack((x1_mesh.flatten(), x2_mesh.flatten()))
y = y_hat(x_mesh, c, v)
y_mesh = y.reshape(x1_mesh.shape)
ax = fig.add_subplot(111, projection='3d')
plt.xlabel('X1')
plt.ylabel('X2')
plt.title('Predicted Values')
ax.plot_surface(x1_mesh, x2_mesh, y_mesh, cmap='viridis')
plt.show()
"""